In [1]:
# 隨機十篇id
# M.1127219109.A.837
# M.1127227032.A.F3B
# M.1127365129.A.D61
# M.1127395683.A.DB9
# M.1127526873.A.CD5
# M.1127533364.A.C32
# M.1127544042.A.E06
# M.1127586620.A.EDB
# M.1127588070.A.525
# M.1127590679.A.006


In [2]:
# 用比較快快的json parser : ujson
try:
    import ujson as json
except ImportError:
    try:
        import simplejson as json
    except ImportError:
        import json

import math


# load tfidf
def readJson2Data(filename):
  f = open(filename, "r", encoding='UTF-8')
  docText = f.read()
  f.close()
  return json.loads(docText)


In [3]:
SAMPLE = 'sample1000_gossiping_dataset_doc_vector.json'
ALLDATA = 'gossiping_dataset_doc_vector.json'
data_tfidf = readJson2Data(ALLDATA)





DOC_NAME = 'data_user-docIDs.json'
docID_name = readJson2Data(DOC_NAME)


In [4]:
# # tools
# def saveSmallData(data):
#     jsObj = json.dumps(data)
    
#     fileObject = open('sample1000.json', 'w')
#     fileObject.write(jsObj)
#     fileObject.close()  
# # saveSmallData(sample)    

In [5]:


sample = {}
counter = 0
for k in data_tfidf.keys():
    if counter == 10:
        break
    print(k)
    data_tfidf[k]
    sample[k] = data_tfidf[k]

    counter = counter + 1

M.1127219109.A.837
M.1127227032.A.F3B
M.1127365129.A.D61
M.1127395683.A.DB9
M.1127526873.A.CD5
M.1127533364.A.C32
M.1127544042.A.E06
M.1127586620.A.EDB
M.1127588070.A.525
M.1127590679.A.006


In [6]:
def calNormalizedLength(id):
    vector = {}
    sum = 0
    for word in data_tfidf[id].keys(): 
        sum = sum + math.pow(data_tfidf[id][word], 2)
    
    sqrt = math.sqrt(sum)

    # 每項除除平方根
    for word in data_tfidf[id].keys():
        vector[word] = data_tfidf[id][word] / sqrt
    return vector


In [7]:
def findCommonWordSet(ida,idb):
    CommonWordSet = set()
    Aset = set( data_tfidf[ida].keys() )
    Bset = set( data_tfidf[idb].keys() )

    for token in Aset:
        if token in Bset:
            CommonWordSet.add(token)
    return CommonWordSet

In [8]:
def cos(ida,idb):
    # calNormalizedLength
    ida_NL = calNormalizedLength(ida)
    idb_NL = calNormalizedLength(idb)

    # findCommonWordSet
    commonWordSet = findCommonWordSet(ida,idb)

    # calCos  內積
    sum = 0
    for word in commonWordSet:
        sum = sum + ida_NL[word] * idb_NL[word]
    
    return sum

In [9]:
cos('M.1127219109.A.837','M.1127227032.A.F3B')

0.026840096055049067

In [10]:
def isUserValid(name):
    try:
        docID_name[name]
        return True
    except:
        return False


In [11]:
def ui():
    name = input("請輸入使用者ID")
    if isUserValid(name) == False:
        print("使用者ID不合法或未在資料庫內 請重新輸入")
        return 0
    else:
        return name
        


In [12]:
import copy
taskDict = {}

def getTargetDocs(targetName):
    return docID_name[targetName]
    

In [15]:
#開啟多個執行緒，同時執行任務，有幾個執行緒就執行幾個任務

import threading
import time
import queue

results = {}

class MyThread(threading.Thread):
    def __init__(self, func):
        threading.Thread.__init__(self)
        self.func = func
    def run(self):
        self.func()

def worker():
    while not q.empty():
        item = q.get()  # 或得任務
        print('Processing : ',item)
        doca , docb = item.split('+')
        try:
            results[docb] = cos(doca,docb)
        except:
            print('error ' + doca + '  '+ docb)

def main(name):
    threads = []
    targetList = getTargetDocs(name)
    # 產生task list
    for doc in targetList:
        for pttdoc in data_tfidf.keys():
            q.put(doc + '+' +pttdoc)

    for i in range(threadNum):   #開啟三個執行緒
        thread = MyThread(worker)
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

if __name__ == '__main__':
    q = queue.Queue()
    threadNum = 23
    
    name = ui()
    if name == 0:
        print("retry")
    else:
        main(name)

KeyboardInterrupt: Interrupted by user

In [ ]:
len(result)

In [ ]:
cos('M.1353503637.A.14B','M.1173516369.A.2C4')
cos('M.1353503637.A.14B','M.1173625796.A.F2A')
cos('M.1353503637.A.14B','M.1173629372.A.C93')

In [ ]:
# data_tfidf['M.1353503637.A.14B']
docID_name['M.1353503637.A.14B']